<a href="https://colab.research.google.com/github/Fatai-soliu/Topic_modelling/blob/main/Soliu_Fata_BerTopic_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#Importing neccessary libraries

import pandas as pd
import seaborn as sns
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [1]:
# Install dependencies

# Create a function that returns the required local i.e. UTF-8

import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [2]:
!pip install bertopic
!pip install flair
!apt-get -qq install -y libfluidsynth1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 17.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 17.6 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (73

In [7]:
# Load and read dataset

url = 'https://github.com/Fatai-soliu/Topic_modelling/raw/master/Scopus%20dataset.csv'

papers = pd.read_csv(url)

In [8]:
papers.head()

,Authors,Author full names,Author(s) ID,Title,Year,Source title,Link,Abstract,Author Keywords
0,Yan L.; Chen Y.; Zheng L.; Zhang Y.,"Yan, Lina (57223593838); Chen, Yile (572219549...",57223593838; 57221954984; 57221968671; 5757867...,Application of computer vision technology in s...,2024,Heritage Science,https://www.scopus.com/inward/record.uri?eid=2...,"In computer artificial intelligence, there is ...",NaN
1,Delcker J.; Heil J.; Ifenthaler D.; Seufert S....,"Delcker, Jan (57192920924); Heil, Joana (58131...",57192920924; 58131602100; 57192168368; 6603394...,First-year students AI-competence as a predict...,2024,International Journal of Educational Technolog...,https://www.scopus.com/inward/record.uri?eid=2...,The influence of Artificial Intelligence on hi...,AI tool; Artificial Intelligence; Chatbot; Hig...
2,Lu H.; Mao Y.; Li J.; Zhu L.,"Lu, Hang (55786333200); Mao, Yuxing (173460731...",55786333200; 17346073100; 57868233800; 5895022...,Multimodal deep learning-based diagnostic mode...,2024,BMC Medical Informatics and Decision Making,https://www.scopus.com/inward/record.uri?eid=2...,Background: Benign paroxysmal positional verti...,BPPV; Deep learning; Feature fusion; Multimoda...
3,Nie W.; Chen Y.; Wang Y.; Wang P.; Li M.; Ning L.,"Nie, Wuzhou (58875023800); Chen, Yong (5723646...",58875023800; 57236468700; 58875378200; 5887628...,Routing networking technology based on improve...,2024,Eurasip Journal on Advances in Signal Processing,https://www.scopus.com/inward/record.uri?eid=2...,Space-air-ground integrated networks comprise ...,Ant colony algorithm; Routing networking proto...
4,Sarumi O.A.; Hahn M.; Heider D.,"Sarumi, Oluwafemi A. (57193360610); Hahn, Maxi...",57193360610; 58817998000; 16743488600,NeuralBeds: Neural embeddings for efficient DN...,2024,Computational and Structural Biotechnology Jou...,https://www.scopus.com/inward/record.uri?eid=2...,The availability of high throughput sequencing...,Artificial intelligence; DNA similarity; Neura...


In [ ]:
# Initiate UMAP
umap_model = UMAP(n_neighbors=15,
                  n_components=5,
                  min_dist=0.0,
                  metric='cosine',
                  random_state=100)
# Clustering model
# hdbscan_model = HDBSCAN(min_cluster_size=5, min_samples = 5,
# metric='euclidean', prediction_data=True)
kmeans_model = KMeans(n_clusters=9)
# Initiate a sentence transformer model
sentence_model = SentenceTransformer("paraphrase-albert-small-v2")
# Initiate a pretrained model
hf_model = pipeline("feature-extraction", model="distilroberta-base")

# Initiate a pretrained embedding model
roberta_model = TransformerDocumentEmbeddings('roberta-base')
# Initiate another pretrained embedding model
glove_embedding = WordEmbeddings('crawl')
document_glove_embeddings = DocumentPoolEmbeddings([glove_embedding])
# Stack the two pretrained embedding models
stacked_embeddings = StackedEmbeddings(embeddings=[roberta_model,
document_glove_embeddings])

# Count vectorizer
vectorizer_model = CountVectorizer(min_df=10)

# Initiate BERTopic
# topic_model = BERTopic(umap_model=umap_model, language="english", calculate_probabilities=True,hdbscan_model=kmeans_model,
#                        embedding_model=stacked_embeddings,min_topic_size=5, n_gram_range=(1, 3),diversity=0.8)#vectorizer_model=vectorizer_model)# Other options for embedding_model are sentence_model, hf_model,roberta_model

# Initiate BERTopic
topic_model = BERTopic(umap_model=umap_model, language="english", calculate_probabilities=True,hdbscan_model=kmeans_model, n_gram_range=(1, 3))
# Run BERTopic model
topics, probabilities = topic_model.fit_transform(papers_nips['abstract_lemmatized'])#abstract_lemmatized

In [ ]:
# Get the list of topics
topic_model.get_topic_info()

In [ ]:

# Get top 10 terms for a topic
topic_model.get_topic(0)

In [ ]:
# Visualize top topic keywords
topic_model.visualize_barchart(top_n_topics=12)

In [ ]:
# Visualize term rank decrease
topic_model.visualize_term_rank()

In [ ]:
# Visualize intertopic distance
topic_model.visualize_topics()

In [ ]:
# Visualize connections between topics using hierachical clustering
topic_model.visualize_hierarchy(top_n_topics=10)

In [ ]:
# Visualize similarity using heatmap
topic_model.visualize_heatmap()

In [ ]:
# Get the topic predictions
topic_prediction = topic_model.topics_[:]
# Save the predictions in the dataframe
papers_nips['topic_prediction'] = topic_prediction
# Take a look at the data
papers_nips.head()

In [ ]:
# New data for the review
new_review = "I like the new headphone. Its sound quality is great."
# Find topics
num_of_topics = 3
similar_topics, similarity = topic_model.find_topics(new_review, top_n=num_of_topics);
# Print results
print(f'The top {num_of_topics} similar topics are {similar_topics}, and the similarities are {np.round(similarity,2)}')

In [ ]:
# Print the top keywords for the top similar topics
for i in range(num_of_topics):
  print(f'The top keywords for topic {similar_topics[i]} are:')
  print(topic_model.get_topic(similar_topics[i]))

In [ ]:
# Save the topic model
topic_model.save("papers_nips_topic_model")
# Load the topic model
my_model = BERTopic.load("papers_nips_topic_model")